In [1]:
%load_ext autotime

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

# Import original dataset

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

# Load Usage data
with open('App_usage_trace.txt') as f:
    lines = f.readlines()#[:100000]
    df = []
    for line in lines:
        data = line.split()
        df.append(data)
usage = pd.DataFrame(df, columns=['uid','timestamp', 'loc', 'app_id','traffic'])
usage['traffic'] = usage['traffic'].astype('float64') / 1e6 # Convert traffic to MB
usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime( x, "%Y%m%d%H%M%S")) 
usage['loc'] = usage['loc'].astype('int64')
usage["date"] = usage["timestamp"].dt.strftime("%m-%d")
# usage['next_timestamp'] = usage['timestamp'].shift(-1)
# usage['time_delta'] = usage['next_timestamp'] - usage['timestamp']
# mask = usage["uid"] != usage["uid"].shift(1)
# usage['time_delta'] = np.where(mask == True, np.nan, usage["time_delta"])
usage.head()

,uid,timestamp,loc,app_id,traffic,date
0,0,2016-04-20 08:13:19,8194,361,0.002,04-20
1,0,2016-04-20 08:13:20,8194,361,0.002,04-20
2,0,2016-04-20 08:13:22,8194,361,0.004,04-20
3,0,2016-04-20 08:13:30,8194,361,0.001,04-20
4,0,2016-04-20 08:13:31,8194,361,0.002,04-20


In [3]:
usage['time_delta'] = usage["timestamp"].diff()
mask = usage["uid"] != usage["uid"].shift(1)
usage['time_delta'] = np.where(mask == True, np.datetime64('NaT'), usage['time_delta'])
usage['time_delta'] = usage['time_delta'].dt.strftime("%H:%M:%S")
usage.head()

,uid,timestamp,loc,app_id,traffic,date,time_delta
0,0,2016-04-20 08:13:19,8194,361,0.002,04-20,NaN
1,0,2016-04-20 08:13:20,8194,361,0.002,04-20,00:00:01
2,0,2016-04-20 08:13:22,8194,361,0.004,04-20,00:00:02
3,0,2016-04-20 08:13:30,8194,361,0.001,04-20,00:00:08
4,0,2016-04-20 08:13:31,8194,361,0.002,04-20,00:00:01


time: 29.3 s


In [4]:
time_delta_count = usage.groupby(['time_delta']).size().reset_index()
time_delta_count.columns = ["time_delta", "count"]
time_delta_count["percent"] = time_delta_count['count']/time_delta_count['count'].sum()
time_delta_count.head(10)

,time_delta,count,percent
0,00:00:00,1096514,0.263
1,00:00:01,1566947,0.376
2,00:00:02,331960,0.080
3,00:00:03,179574,0.043
4,00:00:04,113359,0.027
5,00:00:05,84455,0.020
6,00:00:06,60745,0.015
7,00:00:07,47397,0.011
8,00:00:08,38121,0.009
9,00:00:09,34457,0.008


time: 764 ms


# Import cleaned dataset
## 1st) Cleaned
## 2nd) Remove oscillation

In [5]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

# Load Usage data
with open('rm_oscillated_raw_data.txt') as f:
    lines = f.readlines()#[:100000]
    df = []
    for line in lines:
        data = line.split(",")
        df.append(data)
cleaned_df1 = pd.DataFrame(df, columns=['uid','timestamp', 'loc', 'app_id','traffic'])
cleaned_df1['timestamp'] = cleaned_df1['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
cleaned_df1["traffic"] = cleaned_df1['traffic'].astype('float64')
cleaned_df1["date"] = cleaned_df1["timestamp"].dt.strftime("%m-%d")
cleaned_df1.head()

,uid,timestamp,loc,app_id,traffic,date
0,0,2016-04-20 08:13:19,8194,361,0.002,04-20
1,0,2016-04-20 08:13:20,8194,361,0.002,04-20
2,0,2016-04-20 08:13:22,8194,361,0.004,04-20
3,0,2016-04-20 08:13:30,8194,361,0.001,04-20
4,0,2016-04-20 08:13:31,8194,361,0.002,04-20


time: 1min 32s


# Import second cleaned dataset
## 1st) Remove oscillation
## 2nd) cleaned

In [6]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)

# Load Usage data
with open('rm_oscillated_cleaned_data.txt') as f:
    lines = f.readlines()#[:100000]
    df = []
    for line in lines:
        data = line.split(",")
        df.append(data)
cleaned_df2 = pd.DataFrame(df, columns=['uid','timestamp', 'loc', 'app_id','traffic'])
cleaned_df2['timestamp'] = cleaned_df2['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
cleaned_df2["traffic"] = cleaned_df2['traffic'].astype('float64')
cleaned_df2["date"] = cleaned_df2["timestamp"].dt.strftime("%m-%d")
cleaned_df2.head()

,uid,timestamp,loc,app_id,traffic,date
0,0,2016-04-21 08:42:21,8194,342,0.030,04-21
1,0,2016-04-21 08:44:12,8194,1,0.008,04-21
2,0,2016-04-21 08:44:47,8194,857,0.027,04-21
3,0,2016-04-21 08:44:48,8194,857,0.002,04-21
4,0,2016-04-21 08:44:49,8194,31,0.009,04-21


time: 1min 28s


In [7]:
# print("="*50)
# print("Original Dataset has {} rows".format(usage.shape[0]))
print("-"*50)
print("First cleaned Dataset has {} rows".format(cleaned_df1.shape[0]))
print("-"*50)
print("Second cleaned Dataset has {} rows".format(cleaned_df2.shape[0]))
print("="*50)

--------------------------------------------------
First cleaned Dataset has 4171950 rows
--------------------------------------------------
Second cleaned Dataset has 3970225 rows
time: 675 µs


# Analysis part 
## descritive analysis

### Orignial Dataset

In [8]:
user_count = usage.groupby(["uid"]).size().reset_index()
user_count.columns = ['uid','count']
user_count.describe()

,count
count,871.000
mean,4789.839
std,38407.519
min,4.000
25%,647.500
50%,1659.000
75%,3534.000
max,1098748.000


time: 1.11 s


In [9]:
usage_rm_user = usage[(usage["uid"] != str(942)) & (usage["uid"] != str(772))]
usage_rm_user_count = usage_rm_user.groupby(["uid"]).size().reset_index()
usage_rm_user_count.columns = ['uid','count']
usage_rm_user_count.describe()

,count
count,869.000
mean,3273.794
std,6321.116
min,4.000
25%,646.000
50%,1657.000
75%,3521.000
max,90381.000


time: 3.52 s


### First Cleaned

In [10]:
cleaned_user_count1 = cleaned_df1.groupby(["uid"]).size().reset_index()
cleaned_user_count1.columns = ['uid','count']
cleaned_user_count1.describe()

,count
count,871.000
mean,4789.839
std,38407.519
min,4.000
25%,647.500
50%,1659.000
75%,3534.000
max,1098748.000


time: 1.12 s


In [11]:
cleaned_user_count2 = cleaned_df2.groupby(["uid"]).size().reset_index()
cleaned_user_count2.columns = ['uid','count']
cleaned_user_count2.describe()

,count
count,697.000
mean,5696.162
std,42888.153
min,200.000
25%,820.000
50%,1975.000
75%,4163.000
max,1098748.000


time: 1.57 s


In [12]:
cleaned_df2_rm = cleaned_df2[(cleaned_df2["uid"] != str(942)) & (cleaned_df2["uid"] != str(772))]
cleaned_user_count2_rm = cleaned_df2_rm.groupby(["uid"]).size().reset_index()
cleaned_user_count2_rm.columns = ['uid','count']
cleaned_user_count2_rm.describe()

,count
count,695.000
mean,3803.168
std,6939.822
min,200.000
25%,818.000
50%,1971.000
75%,4150.000
max,90381.000


time: 2.7 s


In [13]:
loc_count_orignal = usage.groupby(["loc"]).size().reset_index()
loc_count_orignal.columns = ['loc','count']
loc_count_orignal.to_csv('original_loc_count.csv')

time: 309 ms


In [14]:
loc_count2 = cleaned_df2.groupby(["loc"]).size().reset_index()
loc_count2.columns = ['loc','count']
loc_count2.to_csv('cleaned_loc_count.csv')

time: 852 ms


In [15]:
loc_count_orignal["count"].describe()

count    6739.000
mean      619.076
std      1245.686
min         1.000
25%        48.000
50%       211.000
75%       636.000
max     24386.000
Name: count, dtype: float64

time: 25.2 ms


In [16]:
loc_count2["count"].describe()

count     6022.000
mean       659.287
std       5110.192
min          1.000
25%         34.000
50%        141.000
75%        426.000
max     186531.000
Name: count, dtype: float64

time: 64.5 ms


In [17]:
required_count = 10

print("Original dataset: There are {} base station that has less than {} total counts ".format((loc_count_orignal[loc_count_orignal["count"] < required_count].shape[0]), required_count))
print("Cleaned dataset: There are {} base station that has less than {} total counts ".format((loc_count2[loc_count2["count"] < required_count].shape[0]), required_count))


Original dataset: There are 737 base station that has less than 10 total counts 
Cleaned dataset: There are 737 base station that has less than 10 total counts 
time: 7.69 ms


# Unique base station

In [18]:
print("="*50)
print("Original Dataset has {} unique base station".format(len(np.unique(usage["loc"]))))
print("-"*50)
print("First cleaned dataset has {} unique base station".format(len(np.unique(cleaned_df1["loc"]))))
print("-"*50)
print("Second cleaned dataset has {} unique base station".format(len(np.unique(cleaned_df2["loc"]))))
print("="*50)

Original Dataset has 6739 unique base station
--------------------------------------------------
First cleaned dataset has 6397 unique base station
--------------------------------------------------
Second cleaned dataset has 6022 unique base station
time: 10.9 s


# Show different rows

In [29]:
# df_diff = pd.concat([cleaned_df1,cleaned_df2]).drop_duplicates(keep=False)
# df_diff

time: 400 µs


# Index 11150-11162

In [25]:
# cleaned_df1.loc[11150:11162,:]

,uid,timestamp,loc,app_id,traffic,date
11150,102,2016-04-21 16:59:02,5025,1285,0.001,04-21
11151,102,2016-04-21 17:02:51,5025,260,0.001,04-21
11152,102,2016-04-21 17:04:21,5025,33,0.002,04-21
11153,102,2016-04-21 17:04:23,5025,93,0.002,04-21
11154,102,2016-04-21 17:04:29,5025,93,0.001,04-21
11155,102,2016-04-21 17:05:28,5045,33,0.001,04-21
11156,102,2016-04-21 17:05:29,5025,33,0.002,04-21
11157,102,2016-04-21 17:05:29,5045,33,0.001,04-21
11158,102,2016-04-21 17:05:30,5045,33,0.002,04-21
11159,102,2016-04-21 17:05:54,5045,33,0.038,04-21


time: 43.1 ms


In [26]:
# cleaned_df2.loc[11150:11162,:]

,uid,timestamp,loc,app_id,traffic,date
11150,103,2016-04-21 19:33:59,4471,65,0.001,04-21
11151,103,2016-04-21 19:34:05,3983,65,0.002,04-21
11152,103,2016-04-21 19:34:15,3983,136,0.006,04-21
11153,103,2016-04-21 19:34:17,3983,136,0.002,04-21
11154,103,2016-04-21 19:34:17,4471,136,0.003,04-21
11155,103,2016-04-21 19:34:18,3983,136,0.002,04-21
11156,103,2016-04-21 19:34:19,3983,136,0.009,04-21
11157,103,2016-04-21 19:34:20,3983,136,0.008,04-21
11158,103,2016-04-21 19:34:20,3983,322,0.002,04-21
11159,103,2016-04-21 19:34:20,4471,322,0.001,04-21


time: 31.9 ms


# Index 105992-106015

In [27]:
# cleaned_df1.loc[105992:106015,:]

,uid,timestamp,loc,app_id,traffic,date
105992,152,2016-04-20 07:52:35,924,1,0.004,04-20
105993,152,2016-04-20 07:57:02,924,47,0.001,04-20
105994,152,2016-04-20 07:57:03,924,47,0.003,04-20
105995,152,2016-04-20 07:59:33,924,315,0.001,04-20
105996,152,2016-04-20 08:06:11,2370,10,0.001,04-20
105997,152,2016-04-20 08:06:31,2370,1,0.001,04-20
105998,152,2016-04-20 08:07:17,2370,42,0.005,04-20
105999,152,2016-04-20 08:08:51,2370,315,0.001,04-20
106000,152,2016-04-20 08:08:58,2370,93,0.000,04-20
106001,152,2016-04-20 08:09:00,2370,1,0.000,04-20


time: 37.6 ms


In [28]:
# cleaned_df2.loc[105992:106015,:]

,uid,timestamp,loc,app_id,traffic,date
105992,171,2016-04-22 13:09:15,3483,1420,0.000,04-22
105993,171,2016-04-22 13:09:21,4329,1420,0.012,04-22
105994,171,2016-04-22 13:09:23,4329,1420,0.004,04-22
105995,171,2016-04-22 13:09:24,4329,1420,0.006,04-22
105996,171,2016-04-22 13:09:30,4329,1420,0.001,04-22
105997,171,2016-04-22 13:09:34,4329,1420,0.006,04-22
105998,171,2016-04-22 13:09:37,4329,1420,0.010,04-22
105999,171,2016-04-22 13:09:49,3483,1420,0.137,04-22
106000,171,2016-04-22 13:09:50,3483,1420,0.000,04-22
106001,171,2016-04-22 13:09:50,4329,1420,0.006,04-22


time: 24.7 ms


#  Analysis on number to clean

In [24]:
# group uid and date, purpose is to see the number of request that a user made for each day
user_usage_perday = usage.groupby(["uid", "date"]).size().reset_index()
# rename columns for easy slicing
user_usage_perday.columns = ["uid", "date", "count"]
# number of users that does not have X counts per day
# change this
required_count = 10

# slice dataset to only remain those rows that has less than 20 count
df_less10day = user_usage_perday[user_usage_perday["count"] < required_count]
# group dataset with uid to get the count of the occurance of the uid, so we can determine which users have just 1 day or more
uid_less10day = df_less10day.groupby(["uid"]).size().reset_index()
# rename columns for easy slicing
uid_less10day.columns = ["uid", "count"]
print("="*100)
print("Current condition: remove user that has less than {} records per day".format(required_count))
print("="*100)
print("There are {} users that does not meet the requirement". format(uid_less10day.shape[0]))
print("-"*100)
print("There are {} users that have less than {} records for exact 1 day".format((uid_less10day.shape[0]- uid_less10day[uid_less10day["count"] != 1].shape[0]), required_count))
print("-"*100)
print("There are {} users that have less than {} records for more than 1 day".format(uid_less10day[uid_less10day["count"] != 1].shape[0], required_count))
print("-"*100)
print("Under current condition, {} rows will be removed, {:.5%} of the entire dataset ".format(df_less10day["count"].sum(), df_less10day["count"].sum()/414171950))
print("="*100)

Current condition: remove user that has less than 10 records per day
There are 200 users that does not meet the requirement
----------------------------------------------------------------------------------------------------
There are 140 users that have less than 10 records for exact 1 day
----------------------------------------------------------------------------------------------------
There are 60 users that have less than 10 records for more than 1 day
----------------------------------------------------------------------------------------------------
Under current condition, 1263 rows will be removed, 0.00030% of the entire dataset 
time: 1.64 s


# Threshold justification